In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A6_019, input_features_A6_019 = pre_models.Input_A6_019('weights/')
    
    outputs = ['Input_A6_019']

    preprocessings = ['min_max_normalization']

    models = [model_A6_019]

    input_features = [input_features_A6_019]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=5
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_016', 'Input_A6_018', 'Input_A2_019', 'Input_A5_016',
       'Input_A5_018'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.00848668158064657
[0.009058779069149557, 0.008685229346375526, 0.00792958914540688, 0.008776851650370076, 0.007104436615507508, 0.007176593275332974, 0.009128721761523465, 0.008654157378349457, 0.009590760417769921, 0.008761697146680332]


test_r2_ave: 0.573286301420308
[0.5620996437762211, 0.547229567457491, 0.6900993930691524, 0.514489399505312, 0.6866987660642252, 0.6844980431815251, 0.504061145102844, 0.5552850729569553, 0.4306687920607808, 0.5577331910285741]


## Linear Regression Baseline

In [5]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.008211383834753831
test_rmse: 0.008484609156697588
test_r2: 0.5902412124594325


## Lasso

In [6]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.008357584368790753
[0.009133612995377278, 0.008401965664511487, 0.008360309115069287, 0.0079248005860788, 0.007920355237441388, 0.007269583594219113, 0.008664188991591125, 0.008184205750250476, 0.008548610577551753, 0.009168211175816805]


test_r2_ave: 0.589662801731906
[0.5548348330989297, 0.5762816418576069, 0.6555186409666978, 0.6041798614364363, 0.6106034146512781, 0.6762688727993693, 0.5532505480384449, 0.6022728781080777, 0.5476756441726937, 0.515741682189526]


## ElasticNet

In [7]:
ElasticNet_rmse_ave(skew_data[title],y,[0.06],0.01)

test_rmse_ave: 0.008500144100729074
[0.009239720863357928, 0.00855570446120569, 0.008723719546023943, 0.00810712304466555, 0.007925049879982009, 0.00754658350969298, 0.00877794051312177, 0.008450671815003561, 0.008519105980184852, 0.009155821394052448]


test_r2_ave: 0.5762248988118045
[0.544431523634133, 0.5606334149045594, 0.6249195351771477, 0.585757427619706, 0.6101416627476327, 0.6511279685717783, 0.5414428587716036, 0.5759524077218412, 0.5507925524357526, 0.5170496365338904]


## SVR

In [8]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.00832192196962971
test_rmse: 0.00853103281802025
test_r2: 0.5846668707032916


## XGB

In [9]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.35,
              reg_alpha =  0.05,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008468798002314306
[0.009219810871358914, 0.008367012772851368, 0.00855466200712067, 0.008356555596579225, 0.007635318127563188, 0.007618575553007027, 0.009083049699047703, 0.008463458302336276, 0.008562927448375876, 0.008826609644902818]


test_r2_ave: 0.57889444121388
[0.5463927500226708, 0.5797997176624095, 0.6393160927236361, 0.5598752262778078, 0.6381262415512368, 0.6444399594078898, 0.5090112113397509, 0.5746682065865962, 0.5461593060315195, 0.5511557005352824]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 7,
            min_samples_leaf =3,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008684580834261145
[0.00948147826998762, 0.00868495326524994, 0.009033860644439293, 0.00829344777636391, 0.00768248733180003, 0.007201134644908318, 0.009189599518867303, 0.00883100723126776, 0.009075970371371266, 0.009371869288356007]


test_r2_ave: 0.5566273316471476
[0.5202797286402128, 0.547258351817449, 0.5977761405801938, 0.5664976765584185, 0.6336412883400799, 0.682336547132161, 0.4974244401387149, 0.5369236405715301, 0.490146922797003, 0.4939885798957123]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-06)
elastic_mod=ElasticNet(alpha=[0.06], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'dart', 
              n_estimators = 80, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.35,
              reg_alpha =  0.05,
              reg_lambda = 0.7)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 7,
            min_samples_leaf =3,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.008323010560092594
[0.009081632820874307, 0.008353183820785204, 0.008446294285529963, 0.007985672799449178, 0.007670869348904805, 0.007178157578979737, 0.00865901419698751, 0.008275948886684663, 0.008513978964245161, 0.009065352898485425]


test_r2_ave: 0.5931625635295811
[0.5598873611362509, 0.581187579157131, 0.648396270436675, 0.5980757367553544, 0.634748515906008, 0.6843604863681545, 0.5537840420028146, 0.5933060339098821, 0.5513330788052744, 0.5265465308182651]


In [10]:

stack_mod = StackingRegressor(regressors=[vote_mod, xgb_mod], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.008357579889323296
[0.009133615722984174, 0.008401961899505442, 0.008360311255319886, 0.007924804696918704, 0.007920351197361643, 0.007269553226378405, 0.008664195984286931, 0.008184202809939038, 0.008548600093301136, 0.009168202007237605]


test_r2_ave: 0.5896632288797754
[0.5548345672160286, 0.5762820216025109, 0.6555184645912746, 0.6041794507879601, 0.6106038119043831, 0.6762715774920499, 0.5532498269128364, 0.6022731638881353, 0.54767675365802, 0.5157426507445547]


In [11]:
ave(skew_data[title],y,0.15,stack_mod,0.7,vote_mod,0.15,lasso_mod)

test_rmse_ave: 0.008324742136434127
[0.009087246558559412, 0.00835893133842598, 0.008412132533500353, 0.00795899634582444, 0.007739222696490872, 0.007195602390337233, 0.008648802280344545, 0.008240194926260883, 0.008515375004182962, 0.009090917290414587]


test_r2_ave: 0.5929686761921156
[0.5593430888518117, 0.5806110421245645, 0.6512347007484944, 0.6007565391750285, 0.6282101708079113, 0.6828244480307053, 0.554835901511161, 0.5968124620293904, 0.55118593061631, 0.5238724780257775]


In [12]:
name='Input_A6_019'
test_pred=[]
seed=[3,4,5,7,6,8]
stack_w=0.15
vote_w=0.7
model_w=0.15
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
